In [ ]:
!pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 22.0 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm

from datasets import load_dataset
from transformers import GPT2Tokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
hf_dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
text = " ".join(hf_dataset["text"])

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
# Tokenize and get indices
tokens = tokenizer.encode(text)
print(f"Total tokens: {len(tokens)}")
print("Example tokens:", tokens[:20])

Token indices sequence length is longer than the specified maximum sequence length for this model (2428601 > 1024). Running this sequence through the model will result in indexing errors


Total tokens: 2428601
Example tokens: [220, 796, 569, 18354, 7496, 17740, 6711, 796, 220, 198, 220, 220, 2311, 73, 13090, 645, 569, 18354, 7496, 513]


In [ ]:
# Prepare training data
seq_length = 5
input_sequences = []
output_tokens = []

# For each position, the first seq_length tokens are inputs, and the next token is the target
for i in range(len(tokens) - seq_length):
  input_sequences.append(tokens[i: i + seq_length])
  output_tokens.append(tokens[i + seq_length])

input_sequences = torch.tensor(input_sequences, dtype=torch.long)
output_tokens = torch.tensor(output_tokens, dtype=torch.long)

In [ ]:
print("Input sequence example:", input_sequences[1])
print("Corresponding target token id:", output_tokens[1])
print("Decoded target word:", tokenizer.decode([output_tokens[1].item()]))

Input sequence example: tensor([  796,   569, 18354,  7496, 17740])
Corresponding target token id: tensor(6711)
Decoded target word:  III


In [ ]:
class NextWordDataset(Dataset):
  def __init__(self, inputs, targets):
      self.inputs = inputs
      self.targets = targets

  def __len__(self):
    return len(self.inputs)

  def __getitem__(self, idx):
    return self.inputs[idx], self.targets[idx]

BATCH_SIZE = 512

dataset = NextWordDataset(input_sequences, output_tokens)
dataloader = DataLoader(
  dataset,
  batch_size=BATCH_SIZE,
  shuffle=True,
  num_workers=0,
  pin_memory=True
)

In [ ]:
len(dataloader), next(iter(dataloader))

(4744,
 [tensor([[  635, 15342, 19627,   955,  1443],
          [  654,   705, 39405,   373,   257],
          [  837, 47098,   287,   281,  6980],
          ...,
          [24768,    71,   382,   290,  9179],
          [  764,  5856,  2159,  1810,   314],
          [ 3700,  2297,  3245,   705,    82]]),
  tensor([  355,  4838,   287,  7068,    31,   262,  1844,  3194, 21624,   663,
            362,  3597,  3825,  3884,   373,  2057,   262,  3648, 11520,   428,
            796,  7685,  6032,   837,  8833,  7028,  7403,  8211,   357,   329,
            286,   566,   625,  5449,   695,  1058,   262,   837,  1912, 31636,
           4492,   220,  7484,   257,   717,  3932,   837,   505,   837, 21791,
           5103,  9520, 11418,   407,   262,  1365,   547,   287,   550,   357,
            220, 25892,   764,   198,  9413,   286,   198,   287,   796,   837,
          29732,  4661,   262,   287, 28407,   453,  4340, 26071,   260,   262,
            837,   373,  3724, 21768,   764,  3124,  9

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, dropout=0.1, feedforward_dim=256):
      super().__init__()
      # Masked Multi-head Self-Attention
      self.self_attn = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, dropout=dropout)
      self.norm1 = nn.LayerNorm(embed_dim)
      self.norm2 = nn.LayerNorm(embed_dim)
      self.dropout = nn.Dropout(dropout)
      # Feed forward Network
      self.feed_forward = nn.Sequential(
          nn.Linear(embed_dim, feedforward_dim),
          nn.ReLU(),
          nn.Linear(feedforward_dim, embed_dim),
          nn.Dropout(dropout)
      )

    def forward(self, x, attn_mask=None):
      # x shape: (seq_length, batch_size, embed_dim)

      # Multi-Head Self-attention
      # We have to apply a target mask to prevent the model from viewing future tokens
      attn_output, _ = self.self_attn(x, x, x, attn_mask=attn_mask) # Q, K, V, mask

      # Residual Addition and Layer Normalization
      x = self.norm1(x + self.dropout(attn_output))

      # Feed-forward network
      ff_output = self.feed_forward(x)

      # Residual Addition and Layer Normalization
      x = self.norm2(x + self.dropout(ff_output))

      return x

In [ ]:
class NextWordDecoderOnlyModel(nn.Module):
    def __init__(self,
                 vocab_size,
                 embed_dim=128,
                 max_seq_length=5,
                 num_heads=8,
                 num_layers=2,
                 dropout=0.1,
                 feedforward_dim=256):
      super().__init__()
      # Embedding layer
      self.token_embedding = nn.Embedding(vocab_size, embed_dim)
      # Positional Encodings (Instead of sinusoidal encodings, we are using learnable posiion embeddings)
      self.position_embedding = nn.Embedding(max_seq_length, embed_dim)
      self.dropout = nn.Dropout(dropout)

      # Create num_layers decoder blocks
      self.layers = nn.ModuleList([
          DecoderBlock(embed_dim, num_heads, dropout, feedforward_dim)
          for i in range(num_layers)
      ])

      # Final linear layer that produces logits
      self.fc_out = nn.Linear(embed_dim, vocab_size)

    def generate_square_subsequent_mask(self, sz):
      # Create the target mask to prevent the model from peeking at future target values
      mask = torch.triu(torch.ones(sz, sz), diagonal=1).bool()
      mask = mask.float().masked_fill(mask, float('-inf'))
      return mask

    def forward(self, x):
      batch_size, seq_length = x.size()

      # Sequential positional encodings that will be multiplied by the learned positional encoding matrix
      positions = torch.arange(seq_length, device=x.device).unsqueeze(0).expand(batch_size, seq_length)

      token_emb = self.token_embedding(x)
      pos_emb = self.position_embedding(positions)
      x = self.dropout(token_emb + pos_emb)  # (batch_size, seq_length, embed_dim)

      # expects inputs of shape (seq_length, batch_size, embed_dim)
      x = x.transpose(0, 1)

      # Create the target mask
      attn_mask = self.generate_square_subsequent_mask(seq_length).to(x.device)

      # Pass the sequences through each decoder layer
      for layer in self.layers:
          x = layer(x, attn_mask=attn_mask)

      # Use the last token's representation to predict the next word.
      last_token = x[-1]  # Get the last token

      last_token += torch.randn_like(last_token) * 0.001

      logits = self.fc_out(last_token)  # Pass through the final linear layer and return the logits for the vocabulary
      return logits

In [ ]:
vocab_size = tokenizer.vocab_size
lr = 0.0005

model = NextWordDecoderOnlyModel(vocab_size,
                                 embed_dim=128,
                                 max_seq_length=seq_length,
                                 num_heads=8,
                                 num_layers=3,
                                 dropout=0.1,
                                 feedforward_dim=256).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.AdamW(params=model.parameters(), lr=lr)

In [ ]:
epochs = 20
model.train()

best_valid_loss = float('inf')
model_path = "next_word_pred_model.pt"

# Training
for epoch in tqdm(range(epochs), desc="Training Progress", colour="#00ff00"):
  total_loss = 0

  for (X, y) in tqdm(dataloader, total=len(dataloader), desc=f"Epoch {epoch + 1} Progress", colour="#005500"):
    X, y = X.to(device), y.to(device)

    # Forward pass
    y_pred = model(X)

    # Calculate the loss
    loss = loss_fn(y_pred, y)
    total_loss += loss.item()

    optimizer.zero_grad()

    # Backprop
    loss.backward()

    # Gradient Clipping
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

    # Optimizer Step
    optimizer.step()

  message = f"Epoch: {epoch + 1} | Loss: {total_loss / len(dataloader)}"

  if total_loss / len(dataloader) < best_valid_loss:
    best_valid_loss = total_loss / len(dataloader)
    torch.save(model.state_dict(), model_path)
    message += " --> STORED"

  print(message)

Training Progress:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1 Progress:   0%|          | 0/4744 [00:00<?, ?it/s]

Epoch: 1 | Loss: 6.26001957325268 --> STORED


Epoch 2 Progress:   0%|          | 0/4744 [00:00<?, ?it/s]

Epoch: 2 | Loss: 5.539704790304322 --> STORED


...

Epoch: 18 | Loss: 4.280227751695688 --> STORED


Epoch 19 Progress:   0%|          | 0/4744 [00:00<?, ?it/s]

Epoch: 19 | Loss: 4.25228465920154 --> STORED


Epoch 20 Progress:   0%|          | 0/4744 [00:00<?, ?it/s]

Epoch: 20 | Loss: 4.224198263571998 --> STORED


In [ ]:
def predict_next_word(model, input_text, tokenizer, seq_length=5):
    model.eval()

    # Tokenize the input text
    tokens = tokenizer.encode(input_text)

    # Pad or truncate to ensure length == seq_length
    if len(tokens) < seq_length:
      tokens = [tokenizer.pad_token_id] * (seq_length - len(tokens)) + tokens
    else:
      tokens = tokens[-seq_length:]

    input_tensor = torch.tensor([tokens], dtype=torch.long).to(device)

    with torch.inference_mode():
      logits = model(input_tensor) # Get the index of the predicted next word

    predicted_id = torch.argmax(logits, dim=-1).item()
    return tokenizer.decode([predicted_id]) # Get the word at index

In [ ]:
def predict_next_words(model, input_text, tokenizer, seq_length=5, num_words=3):
    model.eval()

    # Tokenize the input text
    tokens = tokenizer.encode(input_text)

    # Pad or truncate to ensure length == seq_length
    if len(tokens) < seq_length:
      tokens = [tokenizer.pad_token_id] * (seq_length - len(tokens)) + tokens
    else:
      tokens = tokens[-seq_length:]
    predicted_words = []

    current_tokens = tokens[:]  # copy current tokens

    for i in range(num_words):
      # Use previously generated words
      input_tensor = torch.tensor([current_tokens], dtype=torch.long).to(device)

      with torch.inference_mode():
        logits = model(input_tensor) # Get the index of the predicted next word

      predicted_id = torch.argmax(logits, dim=-1).item()
      predicted_word = tokenizer.decode([predicted_id]) # Get the word at index

      predicted_words.append(predicted_word)
      current_tokens.append(predicted_id)
      current_tokens = current_tokens[-seq_length:] # Only keep the last seq_length elements for model prediction
    return predicted_words

In [ ]:
example_input = "the meaning of"
predicted_word = predict_next_word(model, example_input, tokenizer, seq_length=seq_length)
print("Predicted next token:", predicted_word)

Predicted next token:  the


In [ ]:
example_input = "the man created a"
predicted_sequence = predict_next_words(model, example_input, tokenizer, seq_length=seq_length, num_words=5)
print("Predicted sequence:", predicted_sequence)

Predicted sequence: [' new', ' game', ' .', ' ', '\n']


In [ ]:
#@title Next Sequence Predictor
example_input = "The Great Gatsby is a good" #@param ""
num_words = 5 #@param
predicted_sequence = predict_next_words(model, example_input, tokenizer, seq_length=seq_length, num_words=num_words)
print("Predicted sequence:", predicted_sequence)

Predicted sequence: [' deal', ' of', ' the', ' song', ' ,']
